In [1]:
%pylab inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from tensorflow.python.client import timeline
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
class Simulation:
    

    def create_mask(r, inner_r, outer_r):
        """Return a mask that is true is distance is within the specified range

        arguments:
        r -- matrix containg distances between all individuals: shape() = (n,m)
        inner_r -- minimum distance (exclusive)
        outer_r -- maximum distance (exclusive)

        return matrix shape = (n,m,2)
        """
        mask = tf.logical_and(tf.less(abs_r, outer_r), tf.greater(abs_r, inner_r))
        mask = tf.cast(mask, tf.float32)
        mask = tf.expand_dims(mask, 2)
        mask = tf.tile(mask,(1,1,2))
        mask = tf.cast(mask, tf.bool)
        return(mask)

    def create_mask_1d(r, inner_r, outer_r):
        """Return a mask that is true is distance is within the specified range

        arguments:
        r -- matrix containg distances between all individuals: shape() = (n,m)
        inner_r -- minimum distance (exclusive)
        outer_r -- maximum distance (exclusive)

        return matrix shape = (n,m,1)
        """
        mask = tf.logical_and(tf.less(abs_r, outer_r), tf.greater(abs_r, inner_r))
        mask = tf.cast(mask, tf.float32)
        mask = tf.expand_dims(mask, 2)
        mask = tf.cast(mask, tf.bool)
        return(mask)

    def zone_vec_v(expanded_val, mask, num_particles, zeros):
        """Apply mask to velocities calculate social alignment
        """
        vals = tf.tile(expanded_val, [1, num_particles, 1])
        #take term from vals where mask is true and from zeros when false
        vals = tf.where(mask, vals, zeros)
        return(tf.reduce_sum(vals, 0))

    def zone_vec_x(rel_pos, mask, zeros):
        #take term from rel_pos where mask is true and from zeros when false
        vals = tf.where(mask, rel_pos, zeros)
        return(tf.reduce_sum(vals, 0))

    def draw(window_size, positions, body_size):
        visualization = np.zeros((window_size, window_size, 3), np.uint8)
        Cord=(np.array(positions)).astype(int) # Tranfer particles cordinates to numpy array format to use in plotting functin

        for i in xrange(len(Cord[:,0])):
            if init_state[i] == 0:
                cv2.circle(visualization,tuple(Cord[i,:]), 
                           int(body_size * window_size / window_size), 
                           (255,101,105),-1)
            else:
                cv2.circle(visualization,tuple(Cord[i,:]), 
                           int(body_size * window_size / window_size), 
                           (55,101,255),-1)

        cv2.imshow("visualization", visualization)
        cv2.waitKey(1)
        
    def main_graph():


In [3]:
#---------------------Placeholders input parametes for the graph---------------------------------------------
x = tf.placeholder(dtype=tf.float32)   # Position of particles in the begining of simulation step (input)
v = tf.placeholder(dtype=tf.float32)   # Velocities of particles  in the begining of simulation step (input)
state = tf.placeholder(dtype=tf.float32)
#***************Create the graph to simulate single molecular dynamic (m.d) simulation step---------------------------------------------------------------------------------------------------

expanded_x1 = tf.expand_dims(x, 0)  
expanded_v1 = tf.expand_dims(v, 0)
expanded_state1 = tf.expand_dims(state, 0)

#normalize v1. This will happen at every step in the simulation

expanded_v12 = tf.square(expanded_v1)
expanded_v12_mag = tf.reduce_sum(expanded_v12, 1)
expanded_v1_mag = tf.sqrt(expanded_v12)
expanded_v1_norm = expanded_v1 / expanded_v1_mag    

In [4]:
if PeriodicBoundary: 
    CellPos = [] #Will hold all the neighboring cells 
    for i in range(-1, 2): #Iterates over the neighboring cells
        for i2 in range(-1, 2):
            CellPos.append([i*arena_size, i2*arena_size]) # (0,0) position of each of the neighbouring cells (top left corner)
    AllParticles = tf.expand_dims(CellPos, 1)  + expanded_x1  #Add particle position info to the neighbor cells
    AllParticles = tf.reshape(AllParticles,[-1, 2]) # Reshape to single array of particles coordinates
    expanded_x2 = tf.expand_dims(AllParticles, 1)
    AllParticlesV = tf.tile(expanded_v1_norm, [1, 9, 1])
    AllParticlesV = tf.reshape(AllParticlesV,[-1, 2])
    expanded_v2 = tf.expand_dims(AllParticlesV, 1)
    AllParticlesState = tf.tile(expanded_state1, [1, 9, 1])
    AllParticlesState = tf.reshape(AllParticlesState,[-1, 1])
    expanded_state2 = tf.expand_dims(AllParticlesState, 1)

else:
    expanded_x2 = tf.expand_dims(x, 1)
    expanded_v2 = tf.expand_dims(v, 1)
    expanded_state2 = tf.expand_dims(state, 1)

NameError: name 'PeriodicBoundary' is not defined

In [7]:
rx=tf.subtract(expanded_x1,expanded_x2 )#Distance between every pair of particles in x in every dimension (dx,dy)
rx2=tf.square(rx) # sqar distane for each particle pair in each dimension  (dx^2,dx^2)
r2=tf.reduce_sum(rx2,2) # absolute squar distance between every pair of particles(dx^2+dx^2)
r=tf.sqrt(r2) # absolute distance between every pair of particles
r=tf.maximum(r,tf.ones_like(r)*0.0002)# To avoid division by zero make min distance larger then 0 this add to prevent simulation explosion if particles get too closed

abs_r = tf.abs(r)

zor_r = tf.constant(10.0 * BODY_SIZE, dtype = tf.float32 ) # radius of zone of alignment
zoo_r = tf.constant(30.0 * BODY_SIZE, dtype = tf.float32) # radius of zone of alignment
zoa_r = tf.constant(100.0 * BODY_SIZE, dtype = tf.float32) # radius of zone of alignment

zor_mask = create_mask(abs_r, 0.0, zor_r)
zoo_mask = create_mask(abs_r, zor_r, zoo_r)
zoa_mask = create_mask(abs_r, zoo_r, zoa_r)
state_mask = create_mask_1d(abs_r, zor_r, zoo_r)

if PeriodicBoundary:
    zeros = tf.zeros([NUM_PARTICLES * 9, NUM_PARTICLES, 2], tf.float32)
    zeros_state = tf.zeros([NUM_PARTICLES * 9, NUM_PARTICLES, 1], tf.float32)

else:
    zeros = tf.zeros([NUM_PARTICLES, NUM_PARTICLES, 2], tf.float32)
    zeros_state = tf.zeros([NUM_PARTICLES, NUM_PARTICLES, 1], tf.float32)


r_expanded = tf.expand_dims(r, 2)
r_expanded = tf.tile(r_expanded, (1, 1, 2))
r_unit = rx / r_expanded
    
zor_x = zone_vec_x(r_unit, zor_mask, zeros)
zoo_v = zone_vec_v(expanded_v2, zoo_mask, NUM_PARTICLES, zeros)
zoa_x = zone_vec_x(r_unit, zoa_mask, zeros)
state_active = zone_vec_v(expanded_state2, state_mask, NUM_PARTICLES, zeros_state)
state_ones = zone_vec_v(tf.ones_like(expanded_state2),
                        state_mask, NUM_PARTICLES, zeros_state)

zor_x = tf.multiply(zor_x, tf.constant(1.0))
zoo_v = tf.multiply(zoo_v, tf.constant(3.0))
zoa_x = tf.multiply(zoa_x, tf.constant(-0.5))

zor_sum = tf.reduce_sum(tf.abs(zor_x), 1)
mask_zor_empty = tf.logical_not(tf.greater(zor_sum, tf.constant(0.0)))

zoa_zoo = tf.multiply(tf.reduce_sum(zoa_x, 1), tf.reduce_sum(zoo_v,1))
mask_both_zones = tf.greater(zoa_zoo, tf.constant(0.0))
both_zones = tf.to_float(mask_both_zones)  #should be 0 if only one zone has individs, 1 if both
both_zones_expanded = tf.expand_dims(both_zones, 1)
both_zones_expanded = tf.tile(both_zones_expanded, (1,2))

outer_zones_v = tf.to_float(tf.pow(.5, both_zones_expanded)) * (zoa_x + zoo_v) 

#states
state_active = tf.reduce_sum(state_active, 1)
state_ones = tf.reduce_sum(state_ones, 1)
state_fraction = state_active / state_ones

ones_state = tf.ones_like(state)
zeros_state = tf.zeros_like(state)
state_new = tf.where(tf.less(state_fraction, .5), zeros_state, state)
state_new = tf.where(tf.greater(state_fraction, .5), ones_state, state_new)

vnew = tf.where(mask_zor_empty, outer_zones_v, zor_x)

vnew = tf.minimum(vnew, tf.ones_like(vnew) * 100000.0) #so speed can't blow up

personal_weight = 1 - SOCIAL_WEIGHT
vnew = SOCIAL_WEIGHT*vnew + personal_weight*init_velocities
noise = tf.random_normal(vnew.get_shape(), mean=0.0, stddev=NOISE_STD_DIV, dtype=tf.float32)
vnew = vnew + noise

vnew_mag = tf.square(vnew)
vnew_mag = tf.reduce_sum(vnew_mag, 1) 
vnew_mag = tf.sqrt(vnew_mag)
vnew_mag = tf.expand_dims(vnew_mag, 1)
vnew_mag = tf.tile(vnew_mag, (1, 2)) 
vnew = vnew / vnew_mag

xnew = x + vnew*DT*4.0
 
#-----------if epetitive  boundary conditions are used make sure particle poistion dont exceed cell size-------------------------------------------------------------------
if PeriodicBoundary:
    xnew=tf.mod(xnew + arena_size, arena_size)# repititive boundary conditions make sure the particle never exit the box


#nvidia-smi


In [ ]:
#------------------------------INPUT PARAMETERS-----------------------------------------------------------------
tf.reset_default_graph()
PeriodicBoundary = True   
arena_size = 1000.0                  # Sets the range of the "simulation world" coordinates
DT = tf.constant(.5, tf.float32)   # Time step: time lapse of molecular dynamic simulation step  #was 0.00005
m = tf.constant(0.5, tf.float32)    # mass of particles
NUM_PARTICLES = 1200                # Number of particles to generate
SOCIAL_WEIGHT = 0.5                 # Weighting of social vector vs previous velocity vector
NOISE_STD_DIV = 0.0
BODY_SIZE = 1.5

np.random.seed(190)
init_positions = np.random.rand(NUM_PARTICLES, 2) * arena_size   # initial position of particles
init_velocities = (np.random.standard_normal(size = (NUM_PARTICLES, 2)))       # initial velocities of particles


# make all velocities unit vectors
init_velocities_mag = np.square(init_velocities)
init_velocities_mag = np.sum(init_velocities_mag, 1)  
init_velocities_mag = np.expand_dims(init_velocities_mag, 1)
init_velocities_mag = np.tile(init_velocities_mag, (1, 2))
init_velocities_mag = np.sqrt(init_velocities_mag)
init_velocities = init_velocities / init_velocities_mag


In [ ]:
init_state = np.zeros((NUM_PARTICLES, 1), np.float32) 
init_state[0:600] = 1

In [8]:
#---------------------------Run the graph---------------------------------------------------------------------------

 

WITH_TIMER = False
arena_size_int = int(arena_size)

with tf.Session() as session: #Create graph session
    if WITH_TIMER:
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
    for i in range(100000):   
        if WITH_TIMER:
            [init_positions, init_velocities, init_state] = (
                session.run([xnew,vnew,state_new],feed_dict = {
                    x: init_positions, 
                    v: init_velocities,
                    state: init_state},  
                options=run_options, run_metadata=run_metadata))


            tl = timeline.Timeline(run_metadata.step_stats)
            ctf = tl.generate_chrome_trace_format()

            with open('timeline.json', 'w') as f:
                f.write(ctf)
        else:
            [init_positions, init_velocities, init_state] = (
                session.run([xnew,vnew,state_new],feed_dict = {
                    x: init_positions, 
                    v: init_velocities,
                    state: init_state}))
    #----------------------Plot particles position real time---------------------------------------------       
        if (i%2==0): 
            draw(arena_size_int, init_positions, BODY_SIZE)
            if cv2.waitKey(1) & 0xFF == 27:
                break  #time delay
        sum_state = np.sum(init_state, 0) / NUM_PARTICLES 
        if sum_state == 0 or sum_state == 1:
            print(sum_state)
            break
    session.close() 
cv2.destroyAllWindows()   